## Global Life expectancy


In [1]:
import polars as pl
import polars.selectors as cs
import os


In [2]:
root_file = 'live_expectancy_at_birth'
# if os.path.exists(root_file + '.parquet'):
if False: # re-generates parquet from CSV
    print(f'{"*"*20} Reading {root_file}.parquet  {"*"*20}')
    df = pl.read_parquet(root_file + '.parquet')
    
else:
    print(f'{"*"*20} Reading {root_file}.csv  {"*"*20}')
    df = (
        pl.read_csv(root_file + '.csv', ignore_errors=True)
        .drop('Indicator Name')
        .rename({'Country Code': 'COUNTRY_CODE','Country Name':'COUNTRY_NAME'})
        .sort('COUNTRY_CODE')
    )
    df.write_parquet(root_file + '.parquet')
df_country_codes = (  # use in a join to map country codes to names
    df.select(cs.starts_with('COUNTRY_'))
)
df_transposed = (
    df
    .drop('COUNTRY_NAME')
    .transpose(
        include_header=True, 
        column_names='COUNTRY_CODE', 
        header_name ='YEAR'
    )
    .with_columns(YEAR = pl.col('YEAR').cast(pl.UInt16))
    .with_columns(cs.all().exclude('YEAR').cast(pl.Float32))
    )
    

print(df.shape)
print(df.columns[:8])
print(df.columns[8:])

print(df.head())
print(df_transposed.head())
print(df_country_codes)

******************** Reading live_expectancy_at_birth.csv  ********************
(266, 66)
['COUNTRY_NAME', 'COUNTRY_CODE', '1960', '1961', '1962', '1963', '1964', '1965']
['1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
shape: (5, 66)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ COUNTRY_N ┆ COUNTRY_C ┆ 1960      ┆ 1961      ┆ … ┆ 2020      ┆ 2021      ┆ 2022      ┆ 2023     │
│ AME       ┆ ODE       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ ---       ┆ ---       ┆ f64       ┆ f64     

In [3]:
df_normalized = (
    df_transposed
    .select(
        'YEAR',
        (cs.all().exclude('YEAR')-cs.all().exclude('YEAR').first()).name.suffix('_NORM')
    )
)
df_normalized

YEAR,ABW_NORM,AFE_NORM,AFG_NORM,AFW_NORM,AGO_NORM,ALB_NORM,AND_NORM,ARB_NORM,ARE_NORM,ARG_NORM,ARM_NORM,ASM_NORM,ATG_NORM,AUS_NORM,AUT_NORM,AZE_NORM,BDI_NORM,BEL_NORM,BEN_NORM,BFA_NORM,BGD_NORM,BGR_NORM,BHR_NORM,BHS_NORM,BIH_NORM,BLR_NORM,BLZ_NORM,BMU_NORM,BOL_NORM,BRA_NORM,BRB_NORM,BRN_NORM,BTN_NORM,BWA_NORM,CAF_NORM,CAN_NORM,…,TCD_NORM,TEA_NORM,TEC_NORM,TGO_NORM,THA_NORM,TJK_NORM,TKM_NORM,TLA_NORM,TLS_NORM,TMN_NORM,TON_NORM,TSA_NORM,TSS_NORM,TTO_NORM,TUN_NORM,TUR_NORM,TUV_NORM,TZA_NORM,UGA_NORM,UKR_NORM,UMC_NORM,URY_NORM,USA_NORM,UZB_NORM,VCT_NORM,VEN_NORM,VGB_NORM,VIR_NORM,VNM_NORM,VUT_NORM,WLD_NORM,WSM_NORM,XKX_NORM,YEM_NORM,ZAF_NORM,ZMB_NORM,ZWE_NORM
u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1961,0.165993,0.299179,0.492001,0.27932,-1.030998,1.075001,0.478996,0.751316,0.945,0.389,0.431099,0.595001,0.700001,0.156097,0.991707,0.573002,0.101997,0.819023,0.458,0.286999,0.907001,0.948051,1.644001,0.359001,0.755001,0.48378,-4.445,1.045998,0.289997,0.432003,0.883995,0.666,0.286999,0.560997,0.484001,0.212929,…,0.205002,5.282322,0.348976,0.209,0.456997,0.409,0.361,0.512596,0.398998,0.776855,0.442001,0.293613,0.298836,0.459999,0.208,0.758999,0.758999,0.244999,0.638,0.655998,4.255264,0.126999,0.5,0.334999,0.705997,0.739002,0.564999,0.37817,0.414997,0.542999,1.855343,0.584999,1.181,0.595997,0.492001,0.393002,0.473999
1962,0.552994,0.708233,0.958,0.902157,-0.764999,2.081001,0.898994,1.199715,1.895,0.375999,0.853683,0.955002,1.360001,0.125366,0.7239,1.191002,0.23,0.517563,0.868,0.678997,1.785,0.244392,3.259003,0.733002,1.592003,0.20771,1.352001,1.402,0.663998,0.855,1.565994,0.103001,0.625,1.092999,0.990002,0.233902,…,0.392002,13.479572,0.368591,0.465,0.835999,0.839001,0.719002,0.943848,0.819,1.268559,0.983002,0.665619,0.807697,0.924,1.513,1.548,1.517998,0.618999,1.188,1.013,10.62093,0.228004,0.348785,0.678997,1.357998,1.291,1.118999,-0.075512,0.157997,1.09,4.344185,1.163002,-0.324001,0.206997,0.991001,0.683002,0.960999
1963,0.894997,0.990925,1.402,1.15728,-0.514,3.066002,1.203995,2.185284,2.857998,0.613007,1.26527,1.108002,2.062,0.094635,0.858047,1.834,0.296997,0.34951,1.306,1.018997,1.742001,1.061707,4.815002,1.078003,1.981003,0.45517,1.927002,1.207001,0.996998,1.290001,2.055,1.903999,1.002998,1.627998,1.518002,0.247559,…,0.677002,14.079945,0.963997,0.737999,1.379997,1.256001,1.079002,1.371609,1.263,2.301479,1.455002,0.953617,1.086922,1.127998,2.362999,2.320999,2.319,1.02,1.744999,1.294998,11.274048,0.340004,0.146347,1.024998,1.954998,2.125999,1.604004,-1.30883,-1.149002,1.644001,4.710342,1.708,1.396,-0.031002,1.32,0.957001,1.450001
1964,1.253998,1.366035,1.874001,1.414944,-0.229,3.991001,1.529999,2.775978,3.843998,0.574005,1.660831,1.082001,2.793003,0.063904,1.336342,2.43,0.339001,1.053169,1.726997,1.324997,2.771999,1.873657,6.312,1.443001,2.981003,1.364639,2.451,1.017998,1.335999,1.686001,2.528999,2.545998,1.429001,2.138,2.068001,0.643173,…,0.903999,14.729656,1.372383,1.063999,1.931,1.688999,1.446003,1.799927,1.744999,3.063313,1.936001,1.323353,1.4212,1.473,3.223,2.800999,3.074997,1.479,2.213001,1.584,11.855373,0.429001,0.395126,1.382999,2.504997,2.792,2.048004,1.381386,-0.573002,2.128002,5.154812,-1.110001,2.417,1.445,1.639999,0.521999,1.938999
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,11.969994,19.687603,30.142002,19.370209,25.118,23.054005,12.003998,27.045834,31.945,12.605003

In [5]:
df_pct_norm = (
    df_transposed
    .select(
        'YEAR',
        (
        (cs.all().exclude('YEAR') - cs.all().exclude('YEAR').first()) 
        / cs.all().exclude('YEAR').first()
        )
    )
)
df_pct_norm

YEAR,ABW,AFE,AFG,AFW,AGO,ALB,AND,ARB,ARE,ARG,ARM,ASM,ATG,AUS,AUT,AZE,BDI,BEL,BEN,BFA,BGD,BGR,BHR,BHS,BIH,BLR,BLZ,BMU,BOL,BRA,BRB,BRN,BTN,BWA,CAF,CAN,…,TCD,TEA,TEC,TGO,THA,TJK,TKM,TLA,TLS,TMN,TON,TSA,TSS,TTO,TUN,TUR,TUV,TZA,UGA,UKR,UMC,URY,USA,UZB,VCT,VEN,VGB,VIR,VNM,VUT,WLD,WSM,XKX,YEM,ZAF,ZMB,ZWE
u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1961,0.002592,0.006773,0.015,0.007393,-0.027179,0.019056,0.006644,0.016773,0.018657,0.006055,0.007299,0.009146,0.011176,0.002204,0.014459,0.011266,0.002358,0.01175,0.011812,0.007956,0.020623,0.013691,0.032468,0.005748,0.012899,0.006986,-0.077454,0.015734,0.006692,0.008126,0.01378,0.011342,0.007892,0.010953,0.012231,0.002993,…,0.005228,0.140455,0.005394,0.004801,0.00903,0.007785,0.006922,0.009384,0.012314,0.017894,0.00758,0.006441,0.007213,0.007359,0.004722,0.014941,0.015727,0.005752,0.013981,0.009454,0.101405,0.001876,0.007166,0.005925,0.012055,0.012659,0.00901,0.006027,0.007149,0.010895,0.036421,0.010607,0.02425,0.017811,0.009358,0.007759,0.008861
1962,0.008634,0.016034,0.029208,0.023879,-0.020167,0.036889,0.01247,0.026783,0.037413,0.005853,0.014454,0.01468,0.021713,0.00177,0.010555,0.023416,0.005316,0.007425,0.022386,0.018822,0.040587,0.003529,0.064364,0.011737,0.0272,0.002999,0.023559,0.021088,0.015321,0.016083,0.024412,0.001754,0.017186,0.021339,0.025018,0.003288,…,0.009996,0.358416,0.005697,0.010681,0.016519,0.01597,0.013787,0.017279,0.025276,0.029221,0.016857,0.014602,0.019496,0.014783,0.034348,0.030473,0.031454,0.014532,0.026034,0.014599,0.253102,0.003369,0.004999,0.012009,0.023187,0.022115,0.017844,-0.001204,0.002722,0.02187,0.085277,0.021087,-0.006653,0.006186,0.018849,0.013485,0.017965
1963,0.013974,0.022435,0.042745,0.030632,-0.01355,0.054349,0.0167,0.048785,0.056425,0.009542,0.021422,0.017032,0.032921,0.001336,0.012511,0.036058,0.006865,0.005014,0.033681,0.028247,0.039609,0.015332,0.095094,0.017261,0.033846,0.006572,0.033578,0.018155,0.023005,0.024265,0.032035,0.032425,0.02758,0.031784,0.03836,0.00348,…,0.017263,0.374379,0.014901,0.016951,0.027268,0.023908,0.020691,0.02511,0.038979,0.053013,0.024951,0.02092,0.026235,0.018047,0.053645,0.04569,0.048051,0.023946,0.03824,0.018663,0.268666,0.005023,0.002098,0.018129,0.033381,0.036419,0.025579,-0.02086,-0.019795,0.032985,0.092465,0.030968,0.028664,-0.000926,0.025107,0.018895,0.027107
1964,0.019579,0.030927,0.057136,0.037453,-0.006037,0.070746,0.021222,0.061972,0.075892,0.008935,0.02812,0.016633,0.044592,0.000902,0.019484,0.047776,0.007836,0.01511,0.044539,0.03673,0.063029,0.027057,0.124659,0.023105,0.050931,0.019705,0.042709,0.015312,0.030827,0.031714,0.039424,0.043358,0.039294,0.041742,0.052259,0.009042,…,0.023052,0.391655,0.021213,0.02444,0.038156,0.03215,0.027728,0.032951,0.053855,0.070562,0.033199,0.029031,0.034304,0.023566,0.073169,0.055139,0.063716,0.034722,0.048496,0.022828,0.28252,0.006338,0.005663,0.024461,0.042772,0.047828,0.032659,0.022016,-0.009872,0.042696,0.10119,-0.020126,0.049628,0.043183,0.031194,0.010306,0.036248
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,0.186888,0.445727,0.918992,0.512716,0.662168,0.408665,0.166505,0.603782,0.630688,0.196211,0.29052,0.118334,0.232027,0.170622,0.194057,0.440958,0.437289,0.176368,0.5445,0.668348,0.651319,0.084691,0.589268,0.14337,0.322296,0.071796,0.26465,0.22215,0.564908,0.426,0.192458,0.278061,0.979954,0.311343,-0.203224,0.155068